# 歯科医師国家試験データ活用ノートブック

このノートブックは、`data/`配下の歯科国試JSON（`dental_*.json`）を直接読み込み、画像URLもそのままNotebookやStreamlitで表示します。

In [ ]:
import json
import streamlit as st
from dental_tools import get_question, get_answer

# 質問リスト読み込み
with open('data/dental_questions.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

# 問題選択
sel = st.selectbox('問題番号を選択', [q['number'] for q in questions])
rec = next(q for q in questions if q['number'] == sel)

st.markdown(f"### 問題 {rec['number']}")
st.write(rec['question'])
if rec.get('choices'):
    for i, c in enumerate(rec['choices']):
        st.write(f"{chr(65+i)}. {c}")

# 画像があればURLのまま表示
if rec.get('image_urls'):
    for url in rec['image_urls']:
        if url:
            st.image(url, use_column_width=True)

## ツール定義（dental_tools.py）

`get_question_details(question_number: str) -> dict|None` などの関数は `dental_tools.py` で実装し、Notebook先頭でインポートします。

In [ ]:
# dental_tools.py の例
import json

def get_question_details(question_number: str):
    with open('data/dental_questions.json', 'r', encoding='utf-8') as f:
        questions = json.load(f)
    for q in questions:
        if q['number'] == question_number:
            return q
    return None

def get_question(question_number: str) -> str:
    q = get_question_details(question_number)
    if not q:
        return ''
    s = q['question']
    if q.get('choices'):
        s += '\n' + '\n'.join([f"{chr(65+i)}. {c}" for i, c in enumerate(q['choices'])])
    return s

def get_answer(question_number: str) -> str:
    q = get_question_details(question_number)
    if not q:
        return ''
    return str(q.get('answer', ''))

## 歯科国試版 run_conversation のシステムプロンプト例

In [ ]:
system_prompt = '''\
あなたは歯科医師国家試験の出題管理アシスタントです。
ユーザーが問題文や正答を求めたら、必ず下記の関数を呼び出し、
自力で推測せずに回答してください。
  - get_question(question_number)
  - get_answer(question_number)
ただし「自分の考えで解いてほしい」と指示があれば、関数を使わず自力で解答してください。
'''

## ツール定義ブロック

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_question",
      "description": "問題番号を受け取り、問題文と選択肢を返す",
      "parameters": {
        "type": "object",
        "properties": {
          "question_number": {
            "type": "string",
            "description": "問題番号（例: '118A45', '117B12'）"
          }
        },
        "required": ["question_number"]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "get_answer",
      "description": "問題番号を受け取り、正答を返す",
      "parameters": {
        "type": "object",
        "properties": {
          "question_number": {
            "type": "string",
            "description": "問題番号（例: '118A45', '117B12'）"
          }
        },
        "required": ["question_number"]
      }
    }
  }
]
available_functions = {
  "get_question": get_question,
  "get_answer": get_answer
}